# 1 Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

# 2 Loading the Data & Preprocessing

In [ ]:
# trim tail: to reduce the influence of extreme values
def trim_tail(data, lower_percentile, upper_percentile):
  low, high = np.percentile(data, [lower_percentile, upper_percentile])
  return np.clip(data, low, high)

def do_knn_impute(train_data, test_data, neighbors):
  # knn impute
  knn_imputer     = KNNImputer(n_neighbors=neighbors)
  train_data_filled  = knn_imputer.fit_transform(train_data)
  test_data_filled   = knn_imputer.transform(test_data)

  # transform to df
  columns_lst       = list(train_data.columns)
  train_data     = pd.DataFrame(train_data_filled, columns = columns_lst, index=train_data.index)
  test_data      = pd.DataFrame(test_data_filled, columns = columns_lst, index=test_data.index)
  return train_data, test_data

# load_preprocess:
#  1. load the data
#  2. Split the DE & FR
#  3. KNN imputer
#  4. trim tail on certain features
def load_preprocess(separate_country=False):
  # 1. Load train and set
  X_train     = pd.read_csv('X_train.csv', index_col='ID').drop(columns=["DAY_ID"])
  Y_train     = pd.read_csv('y_train.csv', index_col='ID')
  X_test      = pd.read_csv('X_test.csv', index_col='ID').drop(columns=["DAY_ID"])
  Y_test      = pd.read_csv('y_test_random_final.csv', index_col='ID')

  if separate_country:
      #   Join features and target for preprocessing
      train_df    = X_train.join(Y_train)
      test_df     = X_test.join(Y_test)

      # 2. Split training data into DE and FR datasets | trim tail
      train_fr    = train_df[train_df.COUNTRY=='FR'].sort_index().drop(columns=["COUNTRY"])
      train_de    = train_df[train_df.COUNTRY=='DE'].sort_index().drop(columns=["COUNTRY"])

      #   Split test data into DE and FR datasets | trim tail
      test_fr     = test_df[test_df.COUNTRY=='FR'].sort_index().drop(columns=["COUNTRY"])
      test_de     = test_df[test_df.COUNTRY=='DE'].sort_index().drop(columns=["COUNTRY"])

      # 3. KNN imputer
      print("KNN imputer started ...")
      train_fr, test_fr = do_knn_impute(train_fr, test_fr, 5)
      train_de, test_de = do_knn_impute(train_de, test_de, 5)
      print("KNN imputer end ! ")

      # 4. trim tail
      #   Choose the features that contain outliers
      trim_list = ['DE_CONSUMPTION', 'FR_CONSUMPTION',
      'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT',
      'DE_NET_IMPORT', 'FR_NET_IMPORT','DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD']
      print("Trim tail started, on columns: {}".format(trim_list))

      #   Trim the train data for FR & DE
      trimmed_train_fr = trim_tail(train_fr[trim_list], 1, 99)
      trimmed_train_de = trim_tail(train_de[trim_list], 1, 99)
      trimmed_test_fr  = trim_tail(test_fr[trim_list], 1, 99)
      trimmed_test_de  = trim_tail(test_de[trim_list], 1, 99)

      #   replace original features with trimed features
      train_fr[trim_list] = trimmed_train_fr
      train_de[trim_list] = trimmed_train_de
      test_fr[trim_list] = trimmed_test_fr
      test_de[trim_list] = trimmed_test_de

      #   seperate the features and target
      X_train_fr  = train_fr.drop(columns=['TARGET'])
      X_train_de  = train_de.drop(columns=['TARGET'])
      Y_train_fr  = train_fr[['TARGET']]
      Y_train_de  = train_de[['TARGET']]

      X_test_fr  = test_fr.drop(columns=['TARGET'])
      X_test_de  = test_de.drop(columns=['TARGET'])
      Y_test_fr  = test_fr[['TARGET']]
      Y_test_de  = test_de[['TARGET']]
      print("Proprocessing finished !")

      return [X_train_fr, Y_train_fr, X_train_de, Y_train_de, X_test_fr, Y_test_fr, X_test_de, Y_test_de]

  # If NOT separate country then return full train and test data
  else:
      ohc         = OneHotEncoder(drop='first')
      X_train['COUNTRY']  = ohc.fit_transform(X_train.COUNTRY.values.reshape(-1,1)).toarray()
      X_train.drop(columns=['DAY_ID'], inplace=True)

      X_test['COUNTRY']   = ohc.fit_transform(X_test.COUNTRY.values.reshape(-1,1)).toarray()
      X_test.drop(columns=['DAY_ID'], inplace=True)

      return X_train, Y_train, X_test, Y_test

In [ ]:
X_train_fr, Y_train_fr, X_train_de, Y_train_de, X_test_fr, Y_test_fr, X_test_de, Y_test_de = load_preprocess(separate_country=True)

KNN imputer started ...
KNN imputer end ! 
Trim tail started, on columns: ['DE_CONSUMPTION', 'FR_CONSUMPTION', 'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT', 'DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD']
Proprocessing finished !


# 3 Featur Engineering

## 3.1 Lag items:  built in-week lag items for DE & FR seperately

In [ ]:
from pandas.errors import PerformanceWarning
import warnings

warnings.filterwarnings('ignore', category=PerformanceWarning)

def add_lagged_features(df):
    cols = ['DE_CONSUMPTION', 'FR_CONSUMPTION', 'DE_NET_EXPORT', 'FR_NET_EXPORT',
    'DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD', 'GAS_RET', 'COAL_RET', 'CARBON_RET']

    lagged_features = pd.DataFrame(index=df.index)
    for col in cols:
        for i in range(1, 3):
            lagged_col_name = f'{col}_LAG{i}'
            lagged_features[lagged_col_name] = df[col].shift(i)

    df = pd.concat([df, lagged_features], axis=1)
    df.fillna(df.mean(), inplace=True)

    return df

## 3.2 Consumption Inspirations

In [ ]:
def consumption_insp(df):
  # average commodity price variations
  df['avg_c_p_v']    = df[['GAS_RET', 'COAL_RET',	'CARBON_RET']].sum(axis=1).mean()
  df['avg_c_p_v_ma5']  = df['avg_c_p_v'].rolling(window=5).mean()
  df['avg_c_p_v_ma10']  = df['avg_c_p_v'].rolling(window=10).mean()

  # nuclear ratio trend
  epsilon = 1e-8
  df['nuclear_ratio_fr'] = df['FR_NUCLEAR'] / (df[['FR_GAS', 'FR_COAL', 'FR_HYDRO', 'FR_NUCLEAR', 'FR_SOLAR', 'FR_WINDPOW']].sum(axis=1) + epsilon)
  df['nuclear_ratio_de'] = df['DE_NUCLEAR'] / (df[['DE_GAS', 'DE_COAL', 'DE_HYDRO', 'DE_NUCLEAR', 'DE_SOLAR', 'DE_WINDPOW']].sum(axis=1) + epsilon)

  # new energy transform efficency
  df['hydro_rain_fr'] = df['FR_HYDRO'] / (df['FR_RAIN'].rolling(window=5).mean() + epsilon)
  df['hydro_rain_de'] = df['DE_HYDRO'] / (df['DE_RAIN'].rolling(window=5).mean() + epsilon)

  df['wind_windpow_fr'] = df['FR_WINDPOW'] / (df['FR_WIND'].rolling(window=2).mean() + epsilon)
  df['wind_windpow_de'] = df['DE_WINDPOW'] / (df['DE_WIND'].rolling(window=2).mean() + epsilon)

  # residual_load premium cost
  df['load_premium_cost_fr']  = df['FR_RESIDUAL_LOAD'] * (df['avg_c_p_v'] + epsilon)
  df['load_premium_cost_de']  = df['DE_RESIDUAL_LOAD'] * (df['avg_c_p_v'] + epsilon)
  df['import_cost_fr']  = df['FR_NET_IMPORT'] * (df['avg_c_p_v'] + epsilon)
  df['import_cost_de']  = df['DE_NET_IMPORT'] * (df['avg_c_p_v'] + epsilon)

  # standerdize the new features
  features_to_scale = ['hydro_rain_fr', 'hydro_rain_de', 'wind_windpow_fr', 'wind_windpow_de',
             'load_premium_cost_fr', 'load_premium_cost_de', 'import_cost_fr', 'import_cost_de']
  # initialize scaler
  scaler = StandardScaler()
  df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
  df.fillna(df.mean(), inplace=True)

  return df

# New Data Set for Next Modeling


## FR

In [ ]:
X_train_fr = add_lagged_features(X_train_fr)
X_test_fr = add_lagged_features(X_test_fr)
X_train_fr = consumption_insp(X_train_fr)
X_test_fr = consumption_insp(X_test_fr)

Y_train_fr
Y_test_fr

## DE

In [ ]:
X_train_de = add_lagged_features(X_train_de)
X_test_de = add_lagged_features(X_test_de)
X_train_de = consumption_insp(X_train_de)
X_test_de = consumption_insp(X_test_de)

Y_train_de
Y_test_de

# Modeling ToolKit

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

def metric_kit(y_test, y_pred):
  # show the MSE
  mse = mean_squared_error(y_test, y_pred)
  print(f"[ Default Decision Tree ] MSE: {mse}")

  # show the spearmanr
  spear = spearmanr(y_test, y_pred).correlation
  print('Spearman correlation - default: {:.1f}%'.format(100 * spear))

def modeling_pipeline(X_train, Y_train, model_regressor, param_grid):
  # train valid set split
  X_train, X_val, y_train, y_val = train_test_split(X_train_fr, Y_train_fr, test_size=0.2, random_state=2024)

  # DT model
  model = model_regressor

  # train model
  model.fit(X_train, y_train)

  # test model
  y_pred = model.predict(X_val)

  # show the metrics
  metric_kit(y_val, y_pred)

  # hyperparameter tuning
  grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1)
  grid_search.fit(X_train, y_train)

  # get the best parameters
  print(f"best_params: {grid_search.best_params_}")

  # use best parameter to predict
  best_model = grid_search.best_estimator_
  y_pred_best = best_model.predict(X_val)

  # show the metrics
  metric_kit(y_val, y_pred_best)

  return best_model

# 4 Decision Tree Modeling

In [ ]:
model_regressor = DecisionTreeRegressor(random_state=2024)
param_grid    = {
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 10, 20],
            'min_samples_leaf': [1, 5, 10]
          }
model = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

[ Default Decision Tree ] MSE: 1.999358661781256
Spearman correlation - default: -1.9%
best_params: {'max_depth': 10, 'min_samples_leaf': 10, 'min_samples_split': 2}
[ Default Decision Tree ] MSE: 1.2060422106188151
Spearman correlation - default: 0.9%


# 5 Random Forest Modeling

In [ ]:
model_regressor = RandomForestRegressor(n_estimators=100, random_state=2024)
param_grid    = {
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 10, 20],
            'min_samples_leaf': [1, 5, 10]
          }
model = modeling_pipeline(X_train_fr, Y_train_fr, model_regressor, param_grid)

[ Default Decision Tree ] MSE: 1.0018365896021084
Spearman correlation - default: 21.5%
best_params: {'max_depth': 10, 'min_samples_leaf': 10, 'min_samples_split': 2}
[ Default Decision Tree ] MSE: 0.9674437564374735
Spearman correlation - default: 15.9%


RandomForestRegressor(random_state=2024)

# Submission ToolKit

In [ ]:
Y_test_fr[:] = model.predict(X_test_fr)

Y_test_fr.to_csv('benchmark_qrt_fr.csv', header = True, index = True)